<a href="https://colab.research.google.com/github/Starrysmh/MARK5828-AdvertisingAnalytics-Project/blob/main/Copy_of_5_Image_Processing_Using_Google_CV_revised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What you need for running this notebook!

1.   Google Cloud Platform (GCP) Account & the json_file with keys/credentials 
2.   Images downloaded from Instagram (if you ran the notebook 3. Downloading Images from Instagram, these would be saved in your g-drive/Advertising Analytics/Instagram_Images)
3.   Active Google Drive Account




# The Vision API

In this colab notebook, We will see how to use call the following Google Vision APIs and extract the essential data and store it in a csv format:


1.   Optical Character Recognition
1.   Face Detection
1.   Landmark Detection
1.   Safe Search Detection
2.   Multiple Object Detection
2.   Image Properties Detection
2.   Web Entities Detection
2.   Logo Detection


For more Information please see the website below to understand these apis :
https://cloud.google.com/vision/docs/features-list#image-properties-5



Please mount your google drive when prompted later so that the data can be stored safely and conveniently there . The program expects access to google drive to store the extracted data.

#Installing/Importing the Essential Libraries

In [ ]:
import os
import json
import time
import cv2
import io
from google.colab import files
import pandas as pd
try:
  import colour
except ModuleNotFoundError:
  !pip install colour
  import colour

In [ ]:
!pip install --upgrade google-cloud-vision
!pip install --upgrade google-api-python-client

     |████████████████████████████████| 368kB 4.2MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 


     |████████████████████████████████| 7.0MB 4.2MB/s 
ERROR: earthengine-api 0.1.269 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 2.10.0 which is incompatible.
  Found existing installation: google-auth-httplib2 0.0.4
    Uninstalling google-auth-httplib2-0.0.4:
      Successfully uninstalled google-auth-httplib2-0.0.4
  Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8


In [ ]:
from google.cloud import vision
from google.cloud import vision_v1
#from google.cloud.vision_v1 import enums
#from google.cloud.vision import types
from google.cloud.vision_v1 import types
from google.protobuf.json_format import MessageToJson
import json
from google.colab import drive

# Connecting to Google Cloud

To access the Google Cloud APIs, we need to upload our unique credentials and establish the link to our accounts.


In [ ]:
print('Please upload the Json file with keys/credentials for google cloud apis:')
key_json=files.upload()
client = vision.ImageAnnotatorClient.from_service_account_json(list(key_json.keys())[0])
os.remove(list(key_json.keys())[0])

Please upload the Json file with keys/credentials for google cloud apis:


Saving key_json.json to key_json.json


# Mount the Google Drive

**Please Mount the google drive now!**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Setting up paths

In [ ]:
json_output_path='./drive/MyDrive/Advertising_Analytics_T2_2021/B2C_B2B/gsus4/json_output/'
formatted_json_path='./drive/MyDrive/Advertising_Analytics_T2_2021/B2C_B2B/gsus4/Formatted_Json/'
extracted_vision_data_path='./drive/MyDrive/Advertising_Analytics_T2_2021/B2C_B2B/gsus4/extracted_vision_data/'

# Clearing up the Folders

This section aims to delete all the files in the following g-drive folders and clear them up
1. Advertising Analytics/json_output
2. Advertising Analytics/Formatted_Json
3. Advertising Analytics/extracted_vision_data



In [ ]:
#json_output_path='./drive/My Drive/Advertising Analytics/json_output/'
#formatted_json_path='./drive/My Drive/Advertising Analytics/Formatted_Json/'
# extracted_vision_data_path='./drive/My Drive/Advertising Analytics/extracted_vision_data/'
def delete_all_files():
  try:
    print(f'{len(os.listdir(json_output_path))} files are about to be deleted from the folder {json_output_path}')
    for i in list(os.listdir(json_output_path)):
      os.remove(json_output_path+i)
  except FileNotFoundError:
    print('Nothing to delete: The folder json_output does not exist!!')
  try:
    print(f'{len(os.listdir(formatted_json_path))} files are about to be deleted from the folder {formatted_json_path}')
    for i in list(os.listdir(formatted_json_path)):
      os.remove(formatted_json_path+i)
  except FileNotFoundError:
    print('Nothing to delete: The folder Formatted_Json does not exist!!')
  try:
    print(f'{len(os.listdir(extracted_vision_data_path))} files are about to be deleted from the folder {extracted_vision_data_path}')
    for i in list(os.listdir(extracted_vision_data_path)):
      os.remove(extracted_vision_data_path+i)
  except FileNotFoundError:
    print('Nothing to delete: The folder extracted_vision_data does not exist!!')
  return None

DON'T RUN THE BELOW LINE UNLESS YOU WANT TO CLEAR THE FOLDERS !!

In [ ]:
delete_all_files()

Nothing to delete: The folder json_output does not exist!!
Nothing to delete: The folder Formatted_Json does not exist!!
Nothing to delete: The folder extracted_vision_data does not exist!!


# Resizing Larger Images

We have added a function that checks for the size of the images.As long as there is an image of more than 900 KB, the function will keep on down-sizing that image. 

This should help avoid the exceptions from the Google API that some of you were facing.

In [ ]:
list(os.listdir(img_path))

FileNotFoundError: ignored

In [ ]:
img_path='./drive/MyDrive/Advertising_Analytics_T2_2021/B2C_B2B/gsus4/Instagram_Images/'
def resize_images(img_path=img_path):
  size_dict={i:os.stat(img_path+i).st_size/1024_000 for i in list(os.listdir(img_path))}
  oversized_images={i for i in size_dict.keys() if size_dict[i]>=0.9}
  if len(oversized_images)>0:
    for i in oversized_images:
      img = cv2.imread(img_path+i, cv2.IMREAD_UNCHANGED)
      scale_percent = 80 # percent of original size
      width = int(img.shape[1] * scale_percent / 100)
      height = int(img.shape[0] * scale_percent / 100)
      dim=(width,height)
      resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
      cv2.imwrite(img_path+i,resized)
    time.sleep(5)
    resize_images()
  else:
    return 'Done, huge images have been resized!'

In [ ]:
resize_images()

# Making Call(s) to the Vision API(s)



*  We have defined a function below to make calls to the vision apis and store the outputs. The function takes in 16 images at a time (one batch) and calls the all the apis of interest.
*  The outputs are stored in Json Format. As such in their raw formats, the json files can be difficult to read.
*  We convert these files to a more readable format and save them for future use







Restrictions of Google API Trial Account:


1. Input Restrictions --> Not More than 20 MB (If our batch of 16 images exceeds 20MB, API would throw an error).
2. Output Restrictions --> If the output json_files for a batch is more than 10 MB, API would throw an error).
3. Limited Calls to the API.

In [ ]:
  def get_results_from_vision_api(batch_size=16):
    
    features = [
        {"type_": vision_v1.Feature.Type.TEXT_DETECTION},
        {"type_": vision_v1.Feature.Type.FACE_DETECTION},
        {"type_": vision_v1.Feature.Type.LABEL_DETECTION},
        {"type_": vision_v1.Feature.Type.IMAGE_PROPERTIES},
        {"type_": vision_v1.Feature.Type.LOGO_DETECTION},
        {"type_": vision_v1.Feature.Type.OBJECT_LOCALIZATION},
        {"type_": vision_v1.Feature.Type.LANDMARK_DETECTION},
        {"type_": vision_v1.Feature.Type.SAFE_SEARCH_DETECTION},
        {"type_": vision_v1.Feature.Type.WEB_DETECTION}
    ]
    jpg_list=[i for i in os.listdir(img_path) if i.endswith('jpg')]
    n_batches=[]
    image_batch=[]
    batch_response=[]
    l=list(range(0,len(jpg_list),batch_size))

    #Forming Batches for api calls
    for i in range(len(l)):
      if i!=len(l)-1:
        n_batch=list(range(l[i],l[i+1]))
        #n_batches.append(list(range(l[i],l[i+1])))
        image_batch.append([jpg_list[k] for k in n_batch])
      else:
        #n_batches.append(list(range(l[-1],len(jpg_list))))
        n_batch=list(range(l[-1],len(jpg_list)))
        image_batch.append([jpg_list[k] for k in n_batch])
    
    for j in range(len(image_batch)):
      img_list=image_batch[j]
      requests=[]
      for filename in img_list:
        try:
          with open(img_path+filename,'rb') as image_file:
            image=types.Image(content=image_file.read())
          request=types.AnnotateImageRequest(image=image,features=features)
          requests.append(request)
        except FileNotFoundError:
          continue
          print('Some Images seem to be corrupt! Please check!!')
#        except :
#          continue
      try:
        batch_response=client.batch_annotate_images(requests=requests)
      except:#GCP restricts the output json file size to be not more than 10 MB per batch (16 images) This exception is to ignore such error and continue 
          print('A batch had exceeded Payload output for the trial account! It has been skipped for the time being')
          continue
          
      for r in range(len(batch_response.responses)):
        if batch_response.responses[r].error.message:
          # raise Exception(
          #       '{}\nFor more info on error messages, check: '
          #       'https://cloud.google.com/apis/design/errors'.format(
          #           batch_response.responses[r].error.message))
          continue
        else:
          try:
             with open(json_output_path+img_list[r].split('.jpg')[0]+'.json','w') as fp:
               json.dump(type(batch_response.responses[r]).to_json(batch_response.responses[r]),fp,indent=4) 
          except FileNotFoundError:
            os.mkdir(json_output_path)
            with open(json_output_path+img_list[r].split('.jpg')[0]+'.json','w') as fp:
               json.dump(type(batch_response.responses[r]).to_json(batch_response.responses[r]),fp,indent=4) 

In [ ]:
get_results_from_vision_api()

A batch had exceeded Payload output for the trial account! It has been skipped for the time being


In [ ]:
json_list=[i for i in os.listdir(json_output_path)]
print(f'We have {len(json_list)} json files generated from the vision API')

We have 1394 json files generated from the vision API


Formatting our json outputs

In [ ]:
def beautify_json(input_json,src_path=json_output_path,dest_path=formatted_json_path):

    json_dict={}
    with open(src_path+'/'+input_json) as json_inp:
        json_data = json.load(json_inp)
    json_file_name=input_json.split('.json')[0]
    json_dict[json_file_name]=json.loads(json_data)

    try:
      with open(dest_path+'/'+json_file_name+str('_pretty.json'),'w') as json_inp:
        json.dump(json_dict[json_file_name],json_inp,indent=4)
    except FileNotFoundError:
      os.mkdir(dest_path)
      with open(dest_path+'/'+json_file_name+str('_pretty.json'),'w') as json_inp:
        json.dump(json_dict[json_file_name],json_inp,indent=4)


In [ ]:
#Iterating over the list of json files and formatting them
for i in json_list:
  beautify_json(input_json=i)

IsADirectoryError: ignored

# Extracting the Data from the Json Files


*   We have defined various functions below (hidden intentionally) to read the json files and extract the relevant features
*   We store these features in a data frame and then write it to a csv file which would be saved to g-drive/Advertising Analytics/extracted_vison_data/



In [ ]:
src_path='./drive/My Drive/Advertising Analytics/Formatted_Json/'
dest_path='./drive/My Drive/Advertising Analytics/extracted_vision_data/'

**The following block of code is hidden intentionally! BUT Don't forget to run it**

In [ ]:
#@title
def read_json(json_file):
    with open(json_file,'r') as json_inp:
        json_data=json.load(json_inp)
    return json_data

def OCR_Features_Extract(json_file):
    json_data=read_json(json_file)
    try:
        text_features=json_data['textAnnotations']
        desc=[]
        locale=[]
        for t in range(len(text_features)):
            if t==0:
                locale.append(text_features[t]['locale'])
            desc.append(text_features[t]['description'])
        #locale=locale*len(desc)
        file_name=[json_file.split('.json')[0].split('/')[-1]]*len(locale)
        text_features_dict={'file':file_name,'locale':locale,'ocr_text_description':' '.join(desc)}
        return text_features_dict
    except KeyError:
        print(f"Seems like the {json_file.split('.json')[0].split('/')[-1]} didn't have any texts!")
        return []
    
def Logo_Features_Extract(json_file):
    json_data=read_json(json_file)
    try:
        logo_features=json_data['logoAnnotations']
        desc=[]
        score=[]
        for t in range(len(logo_features)):
            desc.append(logo_features[t]['description'])
            score.append(logo_features[t]['score'])
        file_name=[json_file.split('.json')[0].split('/')[-1]]*len(score)
        logo_features_dict={'file':file_name,'logo_description':desc,'logo_score':score}
        return logo_features_dict
    except KeyError:
        print(f"Seems like the {json_file.split('.json')[0].split('/')[-1]} didn't have any logos!")
        return []


def Label_Features_Extract(json_file):
    json_data=read_json(json_file=json_file)
    try:
        label_features=json_data['labelAnnotations']
        desc=[]
        score=[]
        topicality=[]
        for t in range(len(label_features)):
            desc.append(label_features[t]['description'])
            score.append(label_features[t]['score'])
            topicality.append(label_features[t]['topicality'])
        file_name=[json_file.split('.json')[0].split('/')[-1]]*len(score)
        label_features_dict={'file':file_name,'label_description':desc,'label_score':score,'topicality':topicality}
        return label_features_dict
    except KeyError:
        print(f"Seems like the {json_file.split('.json')[0].split('/')[-1]} didn't have any labels!")
        return []


def Face_Features_Extract(json_file):
    json_data=read_json(json_file)
    try:
        face_features=json_data['faceAnnotations']
        anger=[]
        sorrow=[]
        joy=[]
        surprise=[]

        blurred=[]
        underexposed=[]
        headwear=[]

        detection_confidence=[]

        for t in range(len(face_features)):
            anger.append(face_features[t]['angerLikelihood'])
            sorrow.append(face_features[t]['sorrowLikelihood'])
            joy.append(face_features[t]['joyLikelihood'])
            surprise.append(face_features[t]['surpriseLikelihood'])
            blurred.append(face_features[t]['blurredLikelihood'])
            underexposed.append(face_features[t]['underExposedLikelihood'])
            headwear.append(face_features[t]['headwearLikelihood'])
            detection_confidence.append(face_features[t]['detectionConfidence'])
        file_name=[json_file.split('.json')[0].split('/')[-1]]*len(detection_confidence)
        face_features_dict={'file':file_name,'anger':anger,'sorrow':sorrow,'joy':joy,'surprise':surprise,
        'blurred':blurred,'underexposed':underexposed,'headwear':headwear,'detection_confidence':detection_confidence}
        return face_features_dict
    except KeyError:
        print(f"Seems like the {json_file.split('.json')[0].split('/')[-1]} didn't have any faces!!")
        return []

def Image_Properties_Features_Extract(json_file):
    json_data=read_json(json_file)
    try:
        color_features=json_data['imagePropertiesAnnotation']['dominantColors']['colors']
        color=[]
        pixel_fraction=[]
        score=[]
        for t in range(len(color_features)):
            color.append((color_features[t]['color']['red'],color_features[t]['color']['green'],color_features[t]['color']['blue']))
            pixel_fraction.append(color_features[t]['pixelFraction'])
            score.append(color_features[t]['score'])
        file_name=[json_file.split('.json')[0].split('/')[-1]]*len(score)
        color_features_dict={'file':file_name,'color':color,'pixel_fraction':pixel_fraction,'color_score':score}
        return color_features_dict
    except KeyError:
        print(f"Seems like the {json_file.split('.json')[0].split('/')[-1]} didn't have any colors!!")
        return []

def Landmark_Features_Extract(json_file):
    json_data=read_json(json_file)
    try:
        landmark_features=json_data['landmarkAnnotations']
        desc=[]
        score=[]
        for t in range(len(landmark_features)):
            desc.append(landmark_features[t]['description'])
            score.append(landmark_features[t]['score'])
        file_name=[json_file.split('.json')[0].split('/')[-1]]*len(score)
        landmark_features_dict={'file':file_name,'landmark_description':desc,'landmark_score':score}
        return landmark_features_dict
    except KeyError:
        print(f"Seems like the {json_file.split('.json')[0].split('/')[-1]} didn't have any landmarks!!")
        return []


def Safe_Search_Features_Extract(json_file):
    json_data=read_json(json_file)
    try:
        safe_search_features=json_data['safeSearchAnnotation']

        safe_search_features_dict={'file':json_file.split('.json')[0].split('/')[-1],'adult':[safe_search_features['adult']],
                                   'medical':[safe_search_features['medical']],
                                   'racy':[safe_search_features['racy']],
                                   'spoof':[safe_search_features['spoof']],
                                   'violence':[safe_search_features['violence']]
                                   }
        return safe_search_features_dict
    except KeyError:
        print(f"Seems like the api failed at this classification of {json_file.split('.json')[0].split('/')[-1]}!!")
        return []


def Web_Features_Extract(json_file):
    json_data=read_json(json_file)
    try:
        web_entities=json_data['webDetection']['webEntities']
        desc=[]
        score=[]
        best_guess_labels=[]
        for k in range(len(json_data['webDetection']['bestGuessLabels'])):
            best_guess_labels.append(json_data['webDetection']['bestGuessLabels'][k]['label'])
        for t in range(len(web_entities)):

            try:
              desc.append(web_entities[t]['description'])
              score.append(web_entities[t]['score'])
            except KeyError:
              if len(desc)>len(score):
                desc.pop(-1)
              elif len(score)>len(desc):
                score.pop(-1)
              else:
                continue

        file_name=[json_file.split('.json')[0].split('/')[-1]]*len(score)
        best_guess_labels=best_guess_labels*len(score)
        web_features_dict={'file':file_name,'best_guess_labels':best_guess_labels,'web_description':desc,'web_score':score}
        return web_features_dict
        
    except KeyError:
        print(f"Seems like the api could not associate the {json_file.split('.json')[0].split('/')[-1]} with any web entities")
        return []

def Multiple_Objects_Features_Extract(json_file):
    json_data=read_json(json_file)
    try:
        multiple_objects_features=json_data['localizedObjectAnnotations']
        name=[]
        score=[]
        for t in range(len(multiple_objects_features)):
            name.append(multiple_objects_features[t]['name'])
            score.append(multiple_objects_features[t]['score'])
        file_name=[json_file.split('.json')[0].split('/')[-1]]*len(name)

        multiple_objects_features_dict={'file':file_name,'object_name':name,'object_score':score}
        return multiple_objects_features_dict
    except KeyError:
        print(f"Seems like the {json_file.split('.json')[0].split('/')[-1]} didn't have multiple objects!")
        return []

# def Crop_Hints_Features_Extract(json_file):
#     json_data=read_json(json_file)
#     try:
#         crop_hints_features=json_data['cropHintsAnnotation']['cropHints']
#         crop_confidence=[]
#         crop_importance_fraction=[]
#         for t in range(len(crop_hints_features)):
#             crop_confidence.append(crop_hints_features[t]['confidence'])
#             crop_importance_fraction.append(crop_hints_features[t]['importanceFraction'])
#         file_name=[json_file.split('.json')[0].split('/')[-1]]*len(crop_confidence)
#         crop_hints_features_dict={'file':file_name,'crop_confidence':crop_confidence,'crop_importance_fraction':crop_importance_fraction}
#         return crop_hints_features_dict

#     except KeyError:
#         print(f"Seems like the api could not find much for {json_file.split('.json')[0].split('/')[-1]}!")
#         return []
##Downloading
##Downloading
def ocr_data_download(src_path='./drive/My Drive/Advertising Analytics/Formatted_Json/',dest_path='./drive/My Drive/Advertising Analytics/extracted_vision_data/'):
  ocr_data=pd.DataFrame()
  for j in os.listdir(src_path):
    z=OCR_Features_Extract(json_file=src_path+j)
    if z!=[]:
      z=pd.DataFrame.from_dict(z)
      ocr_data=ocr_data.append(z,ignore_index=True)
  try:
    os.listdir(dest_path)
    ocr_data.to_excel(dest_path+'/'+'ocr_data.xlsx',index=None)
    files.download(dest_path+'ocr_data.xlsx')
  except FileNotFoundError:
    os.mkdir(dest_path)
    ocr_data.to_excel(dest_path+"/"+'ocr_data.xlsx',index=None)
    files.download(dest_path+'ocr_data.xlsx')

def logo_data_download(src_path='./drive/My Drive/Advertising Analytics/Formatted_Json/',dest_path='./drive/My Drive/Advertising Analytics/extracted_vision_data/'):
  logo_data=pd.DataFrame()
  for j in os.listdir(src_path):
    z=Logo_Features_Extract(json_file=src_path+j)
    if z!=[]:
      z=pd.DataFrame.from_dict(z)
      logo_data=logo_data.append(z,ignore_index=True)
  try:
    os.listdir(dest_path)
    logo_data.to_excel(dest_path+'/'+'logo_data.xlsx',index=None)
    files.download(dest_path+'logo_data.xlsx')
  except FileNotFoundError:
    os.mkdir(dest_path)
    logo_data.to_excel(dest_path+"/"+'logo_data.xlsx',index=None)
    files.download(dest_path+'logo_data.xlsx')

def face_data_download(src_path='./drive/My Drive/Advertising Analytics/Formatted_Json/',dest_path='./drive/My Drive/Advertising Analytics/extracted_vision_data/'):
  face_data=pd.DataFrame()
  for j in os.listdir(src_path):
    z=Face_Features_Extract(json_file=src_path+j)
    if z!=[]:
      z=pd.DataFrame.from_dict(z)
      face_data=face_data.append(z,ignore_index=True)
  try:
    os.listdir(dest_path)
    face_data.to_excel(dest_path+'/'+'face_data.xlsx',index=None)
    files.download(dest_path+'face_data.xlsx')
  except FileNotFoundError:
    os.mkdir(dest_path)
    face_data.to_excel(dest_path+"/"+'face_data.xlsx',index=None)
    files.download(dest_path+'face_data.xlsx')

def landmark_data_download(src_path='./drive/My Drive/Advertising Analytics/Formatted_Json/',dest_path='./drive/My Drive/Advertising Analytics/extracted_vision_data/'):
  landmark_data=pd.DataFrame()
  for j in os.listdir(src_path):
    z=Landmark_Features_Extract(json_file=src_path+j)
    if z!=[]:
      z=pd.DataFrame.from_dict(z)
      landmark_data=landmark_data.append(z,ignore_index=True)
  try:
    os.listdir(dest_path)
    landmark_data.to_excel(dest_path+'/'+'landmark_data.xlsx',index=None)
    files.download(dest_path+'landmark_data.xlsx')
  except FileNotFoundError:
    os.mkdir(dest_path)
    landmark_data.to_excel(dest_path+"/"+'landmark_data.xlsx',index=None)
    files.download(dest_path+'landmark_data.xlsx')

def image_properties_data_download(src_path='./drive/My Drive/Advertising Analytics/Formatted_Json/',dest_path='./drive/My Drive/Advertising Analytics/extracted_vision_data/'):
  image_properties_data=pd.DataFrame()
  for j in os.listdir(src_path):
    z=Image_Properties_Features_Extract(json_file=src_path+j)
    if z!=[]:
      z=pd.DataFrame.from_dict(z)
      image_properties_data=image_properties_data.append(z,ignore_index=True)
  try:
    os.listdir(dest_path)
    image_properties_data.to_excel(dest_path+'/'+'image_properties_data.xlsx',index=None)
    files.download(dest_path+'image_properties_data.xlsx')
  except FileNotFoundError:
    os.mkdir(dest_path)
    image_properties_data.to_excel(dest_path+"/"+'image_properties_data.xlsx',index=None)
    files.download(dest_path+'image_properties_data.xlsx')


def safe_search_data_download(src_path='./drive/My Drive/Advertising Analytics/Formatted_Json/',dest_path='./drive/My Drive/Advertising Analytics/extracted_vision_data/'):
  safe_search_data=pd.DataFrame()
  for j in os.listdir(src_path):
    z=Safe_Search_Features_Extract(json_file=src_path+j)
    if z!=[]:
      z=pd.DataFrame.from_dict(z)
      safe_search_data=safe_search_data.append(z,ignore_index=True)
  try:
    os.listdir(dest_path)
    safe_search_data.to_excel(dest_path+'/'+'safe_search_data.xlsx',index=None)
    files.download(dest_path+'safe_search_data.xlsx')
  except FileNotFoundError:
    os.mkdir(dest_path)
    safe_search_data.to_excel(dest_path+"/"+'safe_search_data.xlsx',index=None)
    files.download(dest_path+'safe_search_data.xlsx')

def web_search_data_download(src_path='./drive/My Drive/Advertising Analytics/Formatted_Json/',dest_path='./drive/My Drive/Advertising Analytics/extracted_vision_data/'):
  web_search_data=pd.DataFrame()
  for j in os.listdir(src_path):
    z=Web_Features_Extract(json_file=src_path+j)
    if z!=[]:
      z=pd.DataFrame.from_dict(z)
      web_search_data=web_search_data.append(z,ignore_index=True)
  try:
    os.listdir(dest_path)
    web_search_data.to_excel(dest_path+'/'+'web_search_data.xlsx',index=None)
    files.download(dest_path+'web_search_data.xlsx')
  except FileNotFoundError:
    os.mkdir(dest_path)
    web_search_data.to_excel(dest_path+"/"+'web_search_data.xlsx',index=None)
    files.download(dest_path+'web_search_data.xlsx') 


def multiple_objects_data_download(src_path='./drive/My Drive/Advertising Analytics/Formatted_Json/',dest_path='./drive/My Drive/Advertising Analytics/extracted_vision_data/'):
  multiple_objects_data=pd.DataFrame()
  for j in os.listdir(src_path):
    z=Multiple_Objects_Features_Extract(json_file=src_path+j)
    if z!=[]:
      z=pd.DataFrame.from_dict(z)
      multiple_objects_data=multiple_objects_data.append(z,ignore_index=True)
  try:
    os.listdir(dest_path)
    multiple_objects_data.to_excel(dest_path+'/'+'multiple_objects_data.xlsx',index=None)
    files.download(dest_path+'multiple_objects_data.xlsx')
  except FileNotFoundError:
    os.mkdir(dest_path)
    multiple_objects_data.to_excel(dest_path+"/"+'multiple_objects_data.xlsx',index=None)
    files.download(dest_path+'multiple_objects_data.xlsx')

# def crop_hints_data_download(src_path='./drive/My Drive/Advertising Analytics/Formatted_Json/',dest_path='./drive/My Drive/Advertising Analytics/extracted_vision_data/'):
#   crop_hints_data=pd.DataFrame()
#   for j in os.listdir(src_path):
#     z=Crop_Hints_Features_Extract(json_file=src_path+j)
#     if z!=[]:
#       z=pd.DataFrame.from_dict(z)
#       crop_hints_data=crop_hints_data.append(z,ignore_index=True)
#   try:
#     os.listdir(dest_path)
#     crop_hints_data.to_excel(dest_path+'/'+'crop_hints_data.xlsx',index=None)
#     files.download(dest_path+'crop_hints_data.xlsx')
#   except FileNotFoundError:
#     os.mkdir(dest_path)
#     crop_hints_data.to_excel(dest_path+"/"+'crop_hints_data.xlsx',index=None)
#     files.download(dest_path+'crop_hints_data.xlsx')




def label_data_download(src_path='./drive/My Drive/Advertising Analytics/Formatted_Json/',dest_path='./drive/My Drive/Advertising Analytics/extracted_vision_data/'):
  label_data=pd.DataFrame()
  for j in os.listdir(src_path):
    z=Label_Features_Extract(json_file=src_path+j)
    if z!=[]:
      z=pd.DataFrame.from_dict(z)
      label_data=label_data.append(z,ignore_index=True)
  try:
    os.listdir(dest_path)
    label_data.to_excel(dest_path+'/'+'label_data.xlsx',index=None)
    files.download(dest_path+'label_data.xlsx')
  except FileNotFoundError:
    os.mkdir(dest_path)
    label_data.to_excel(dest_path+"/"+'label_data.xlsx',index=None)
    files.download(dest_path+'label_data.xlsx')

# Downloading the Extracted Data as CSVs

The following section aims to download the data extracted from the API as CSVs. It would automatically be saved in the g-drive as well in the folder g-drive/Advertising Analytics/extracted_vision_data/

Please note that you may not need to run all the following commands. Based on your research question, you may choose to run only selective commands.

Note: You will end up downloading an empty file in case the API didn't give any outputs.

1. Optical Character Recognition (OCR)

In [ ]:
ocr_data_download()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

2. Logo Detection

In [ ]:
logo_data_download()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

3. Face Detection

In [ ]:
face_data_download()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

4. Multiple Object Detection

In [ ]:
multiple_objects_data_download()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 5. Image Properties Detection

In [ ]:
image_properties_data_download()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

6. Web Entities Detection

In [ ]:
web_search_data_download()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

7. Safe Search Detection

In [ ]:
safe_search_data_download()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

8. Landmark Detection

In [ ]:
landmark_data_download()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

9. Label Detection

In [ ]:
label_data_download()